**Data Cleaning for Customer address data sheet from Raw data**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime, date
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading customer Address data  sheet from raw excel file
cust_add = pd.read_excel('Raw_data.xlsx',sheet_name="CustomerAddress")


In [5]:
#check record of data set
cust_add.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [6]:
#Information related to data

cust_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


From the above information we could see that the data types is accurate.lets proceed with the data cleaning process wherever applicable before proceeding with the analysis

In [9]:
print("No of rows: ", (cust_add.shape[0]))
print("No of columns: ", (cust_add.shape[1]))

No of rows:  3999
No of columns:  6


**Numeric and Non Numeric data set**

In [12]:
# select numeric columns
df_numeric = cust_add.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print("The numeric columns are : {}".format(numeric_cols))


#select non numeric columns

df_non_numeric =cust_add.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print("The non numeric columns are: {}".format(non_numeric_cols))

The numeric columns are : ['customer_id' 'postcode' 'property_valuation']
The non numeric columns are: ['address' 'state' 'country']


**Missing Value check**

In [13]:
cust_add.isna().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

There is no missing value in the data set so we could proceed for checking data inconsistency

**Inconsistency Check in Data**

We will check whether there is inconsistent data / typo error data is present in the categorical columns.
The columns to be checked are 'address', 'postcode' ,'state', 'country'

In [14]:
# state

cust_add["state"].value_counts()

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64

Here there are inconsistent data in State column. For New South Wales and Victoria we have two values, one being the full name and the other being their short name. The State names should be standardised and columns with state as New South Wales will be replaced by NSW and columns with state as Victoria will be replaced by VIC.

In [15]:
# Function to replace full state names with their short forms.

def replace_state_name(state_name):
    if state_name=="New South Wales":
        return "NSW"
    elif state_name=="Victoria":
        return "VIC"
    else:
        return state_name
    
 
# Applying the above fuction to state column

cust_add["state"]=cust_add["state"].apply(replace_state_name)

In [16]:
cust_add["state"].value_counts()

state
NSW    2140
VIC    1021
QLD     838
Name: count, dtype: int64

After applying the function there is no inconsistency in state column and it is standardised 

In [17]:
#country

cust_add["country"].value_counts()

country
Australia    3999
Name: count, dtype: int64

There is no inconsistency or typo error in country columns

In [19]:
#postcode

cust_add["postcode"].value_counts()

cust_add[['address','postcode', 'state' , 'country']].drop_duplicates()


,address,postcode,state,country
0,060 Morning Avenue,2016,NSW,Australia
1,6 Meadow Vale Court,2153,NSW,Australia
2,0 Holy Cross Court,4211,QLD,Australia
3,17979 Del Mar Point,2448,NSW,Australia
4,9 Oakridge Court,3216,VIC,Australia
...,...,...,...,...
3994,1482 Hauk Trail,3064,VIC,Australia
3995,57042 Village Green Point,4511,QLD,Australia
3996,87 Crescent Oaks Alley,2756,NSW,Australia
3997,8194 Lien Street,4032,QLD,Australia


The Postcode column looks perfect. There is no inconsistency / typo in the data.

**Duplicate records**


In [20]:
#Return boolean series of duplicate values 
duplicates=cust_add.duplicated()


#display duplicates rows
cust_add[duplicates]

,customer_id,address,postcode,state,country,property_valuation


There is no duplicate records in the data set 

**Exporting the Cleaned Customer Address Data Set to csv**


Currently the Customer Address dataset is clean. Hence we can export the data to a csv to continue our data analysis of Customer Segments by joining it to other tables

In [21]:
cust_add.to_csv('CustomerAddress_Cleaned.csv', index=False)


**Checking for Master Detail Records**:

Checking with the Master Table (CustomerDemographic_Cleaned.csv) containing the entire Customer Data for the Customer IDs which are getting dropped from the Customer Address Dataset.
Basically these are the Customers who have an address but are not a part of the Demographics dataset yet.

In [22]:
cust_demo_detail = pd.read_csv('CustomerDemographic_Cleaned.csv')

In [23]:
cust_demo_detail.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,71
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,44
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,71
3,4,Talbot,NaN,Male,33,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,63
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,47


In [24]:
print("Total Records in Customer_Demographic_Table : {}".format(cust_demo_detail.shape[0]))
print("Total Records in Customer_Address_Table : {}".format(cust_add.shape[0]))
print('In Demographic Table {} records are getting dropped due to data cleaning process in Demographic Table'
      .format(cust_add.shape[0]-cust_demo_detail.shape[0]))

Total Records in Customer_Demographic_Table : 3912
Total Records in Customer_Address_Table : 3999
In Demographic Table 87 records are getting dropped due to data cleaning process in Demographic Table


**Customer Id's in Address table are dropped**

In [25]:
cust_drop = cust_add.merge(cust_demo_detail , left_on = 'customer_id', right_on='customer_id'
                     , how='outer')
cust_drop.head()

,customer_id,address,postcode,state,country,property_valuation,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,060 Morning Avenue,2016.0,NSW,Australia,10.0,Laraine,Medendorp,Female,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,71.0
1,2,6 Meadow Vale Court,2153.0,NSW,Australia,10.0,Eli,Bockman,Male,81.0,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,44.0
2,4,0 Holy Cross Court,4211.0,QLD,Australia,9.0,Talbot,NaN,Male,33.0,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,63.0
3,5,17979 Del Mar Point,2448.0,NSW,Australia,4.0,Sheila-kathryn,Calton,Female,56.0,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,47.0
4,6,9 Oakridge Court,3216.0,VIC,Australia,9.0,Curr,Duckhouse,Male,35.0,1966-09-16,Missing,Retail,High Net Worth,N,Yes,13.0,58.0


In [26]:
cust_drop.isna().sum()

customer_id                              0
address                                  4
postcode                                 4
state                                    4
country                                  4
property_valuation                       4
first_name                              91
last_name                              212
gender                                  91
past_3_years_bike_related_purchases     91
DOB                                     91
job_title                               91
job_industry_category                   91
wealth_segment                          91
deceased_indicator                      91
owns_car                                91
tenure                                  91
Age                                     91
dtype: int64